In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from src.constants import *

from src.utility.personal import *
from src.utility.structure import *
from src.utility.storage import *
from src.utility.math import *

from src.avgs import *
from src.trajectory import *

from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
from src.init_DFs.next_game import NextGameInit

from src.relative_data.power_rank import *
from src.relative_data.sos import *
from src.situational_data.home_away import *


color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

pgModel = PerGameModel()
initPG = PerGameInit()
initNextGame = NextGameInit()

In [2]:
season = 2022
situation = "5on5"
df = primary_csv()
game_date = '4/3/2023'
initial_date = ''
HA = ""

In [3]:
odds = matchup_odds_matrix(df.copy(), season, game_date, initial_date)
mtrx = odds['mtrx']
gp = odds['gp']

pr = power_rank__implied(df.copy(), season, game_date, initial_date, HA)
pr_num = get_number_rank(pr)
pr_m = power_rank__implied_matchup(df.copy(), season, game_date, initial_date, HA)
pr_m_num = get_number_rank(pr_m, False)

sos = sos__implied(df, season, game_date, initial_date, HA)
sos_m = sos__implied_matchup(df, season, game_date, initial_date, HA)
pr_sos = sos__adjust(pr, gp)
pr_m_sos = sos__adjust(pr_m, gp)


odds_h = matchup_odds_matrix_HA(df.copy(), season, game_date, "H", initial_date)
odds_a = matchup_odds_matrix_HA(df.copy(), season, game_date, "A", initial_date)

pr_h = power_rank__implied(df.copy(), season, game_date, initial_date, "H")
pr_a = power_rank__implied(df.copy(), season, game_date, initial_date, "A")
sos_h = sos__implied(df, season, game_date, initial_date, "H")
sos_a = sos__implied(df, season, game_date, initial_date, "A")

h_adv = home_away_adv__implied(df, season, game_date, "H", initial_date)
h_adv_m = home_away_adv__implied_matchup(df, season, game_date, "H", initial_date)
a_adv = home_away_adv__implied(df, season, game_date, "A", initial_date)
a_adv_m = home_away_adv__implied_matchup(df, season, game_date, "A", initial_date)

hi = home_ice__adjust(df, season, game_date, initial_date)
hi_m = home_ice__adjust_matchup(df, season, game_date, initial_date)

In [120]:
odds_h['mtrx']['BOS']['ANA']
#odds_a['mtrx']
odds_h["mtrx"].iloc[0, 2]
#odds_a["mtrx"].iloc[0, 2]
h_adv.mean() - h_adv
pr_m_sos.sort_values(ascending=True).copy()
#pr_m_sos

def sos__roe(pr, gp):
    sos = pd.Series()

    for col in gp.columns:
        sos[col] = (pr.copy() * gp[col]).sum() / gp[col].sum()

    exp_pr = pr.mean() * 2 - sos.copy()

    pr_above_exp = pr.copy() - exp_pr.copy()

    return pr_above_exp.sort_values(ascending=False).copy()

#veg = df.odds.mean() + df.opp_odds.mean() + df.ot_odds.mean()

woe(pr, gp)

BOS    0.126871
CAR    0.115337
TOR    0.100002
TBL    0.085263
CGY    0.073098
FLA    0.068753
COL    0.067686
NYR    0.066089
NJD    0.065305
EDM    0.065232
PIT    0.064793
DAL    0.046044
VGK    0.042490
MIN    0.037683
WSH    0.015166
LAK    0.014226
NYI    0.013374
WPG    0.010174
SEA   -0.001716
OTT   -0.020231
NSH   -0.026837
BUF   -0.027136
STL   -0.040669
VAN   -0.041940
DET   -0.057844
PHI   -0.082530
SJS   -0.092670
CBJ   -0.120073
MTL   -0.126359
ANA   -0.138222
ARI   -0.145369
CHI   -0.160908
dtype: float64

In [114]:
#sos_m = sos_m - sos_m.mean()
pr_m_sos = pr_m_sos.copy() + sos_m
pr_m_sos
odds["mtrx"]["CAR"]["ANA"]
h_adv
df.is_home
sos

PHI    0.425434
MTL    0.423977
CBJ    0.422510
BUF    0.421608
PIT    0.421350
ARI    0.420882
NYI    0.419684
SEA    0.419554
DET    0.419502
ANA    0.418737
NYR    0.418141
CHI    0.417436
VAN    0.417287
BOS    0.417276
CAR    0.416935
NJD    0.416230
FLA    0.415821
WSH    0.415509
OTT    0.415064
TOR    0.413505
LAK    0.412695
TBL    0.412665
STL    0.411829
CGY    0.410700
COL    0.410685
VGK    0.410592
EDM    0.410505
DAL    0.410298
WPG    0.409401
NSH    0.409020
MIN    0.408311
SJS    0.405538
dtype: float64

In [44]:
h_adv.mean()

0.03766211056765104